In [ ]:
import yfinance as yf 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tickers = ["GOOGL", "NVDA", "FB", 
           "PLTR", "WFC", 
           "DIS", "LUV", "PFE", 
           "COKE", "CAT"]

In [ ]:
data = yf.download(tickers, start="2020-01-01", end="2026-01-01")["Close"]

In [ ]:
x_train = data.iloc[data.shape[0]-252:,:]
x_test = data.iloc[:data.shape[0]-252,:]

In [ ]:
E_R = x_train.mean()
E_R.columns = tickers
E_R_ann = pd.DataFrame(x_train.mean()*250)
E_R_ann = E_R_ann.T
dar = x_train.pct_change().iloc[1:,:]+1
gar = pd.DataFrame(np.prod(dar)**(1/float(6))-1)
full_return_ann = (pd.concat([E_R_ann.T, gar], axis=1))
full_return_ann.columns = ["Annualized Arithmetic Returns", "Geometric Averagr Returns"]

In [ ]:
portfolio_returns = []
portfolio_vol = []
sharpe_ratio = []
stock_weights = []
num_assets = len(tickers)
num_portfolios = 10000

cov_daily = x_train.pct_change().cov()
cov_annual = cov_daily*250

In [ ]:
np.random.seed(42)

for i in range(num_portfolios):
  weights = np.random.random(num_assets)
  weights = weights/np.sum(weights)
  returns = np.dot(weights, E_R)
  vol = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
  sr = (returns-1)/vol

  portfolio_returns.append(returns-1)
  sharpe_ratio.append(sr)
  portfolio_vol.append(vol)
  stock_weights.append(weights)

  

In [ ]:
portfolio = {
    "returns": portfolio_returns,
    "volatility": portfolio_vol,
    "sharpe_ratio": sharpe_ratio
}

for c, s in enumerate(tickers):
  portfolio[s+" Weight"] = [w[c] for w in stock_weights]

df = pd.DataFrame(portfolio)
df

In [ ]:
df.plot.scatter(x="volatility", y="returns", c="sharpe_ratio")
plt.xlabel("Volatility (Std Dev)")
plt.ylabel("Expected Return")
plt.title("Efficient Frontier")
plt.show()

In [ ]:
min_vol = df["volatility"].min()
max_sharpe = df["sharpe_ratio"].max()
sharpe_portfolio = df.loc[df["sharpe_ratio"]==max_sharpe]
min_var_portfolio = df.loc[df["volatility"]==min_vol]
df.plot.scatter(x="volatility", y="returns", c="sharpe_ratio")
plt.scatter(x=sharpe_portfolio["volatility"], y=sharpe_portfolio["returns"], c="red", s=200)
plt.scatter(x=min_var_portfolio["volatility"], y=min_var_portfolio["returns"], c="blue", s=200)
plt.xlabel("Volatility (Std Dev)")
plt.ylabel("Expected Return")
plt.title("Efficient Frontier")
plt.show()

In [ ]:
r_ef = pd.concat([min_var_portfolio, sharpe_portfolio], axis=0).T
r_ef.columns = ["Min Var", "Max Sharpe"]
r_ef

In [ ]:
amount_invest = 1000
expected_return = pd.DataFrame(amount_invest * (1+r_ef.iloc[0,:]))
annual_return = (x_test.iloc[x_test.shape[0]-1,:] - x_test.iloc[0,:]) / (x_test.iloc[0,:])
beg_price = x_test.iloc[0,:]
end_price = x_test.iloc[df.shape[0]-1,:]
w = np.array(r_ef.iloc[3:,0])
pct_change = (end_price - beg_price)/(beg_price)+1
roi = np.sum(w * pct_change*amount_invest)
print(expected_return.T)
print(amount_invest)
print(round(roi))